# Import and view the data

In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sps

path = 'C:/Users/Utente/Desktop/RecSys_Course_AT_PoliMi-master'
path1 = path + '/interactions_and_impressions.csv'
path2 = path + '/data_ICM_length.csv'
path3 = path + '/data_ICM_type.csv'


#reading the interaction matrix twice, one for impression discounting, I tried to copy it while making it equal to dummy = URM 
#but it did not work, so I did a dummy soln its fine. Also dummy needs to be read like this cause I need to be sure that 
#"Impressions" is in string format.
dummy =  pd.read_csv(filepath_or_buffer=path1,
                                sep=",",
                                header=None,
                                dtype={"UserID":int, "ItemID":int, "Impressions":str, "Data":int},
                                engine='python')
dummy.columns = ["UserID", "ItemID", "Impressions", "Data"]
#When you read like this row 0 is string of column names so I am dropping them
dummy =dummy.drop(0)

URM_interactions = pd.read_csv(path1)
ICM_length = pd.read_csv(path2)
ICM_type = pd.read_csv(path3)





C:\Users\Utente\AppData\Local\Temp\ipykernel_21088\342945934.py:23: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  URM_interactions = pd.read_csv(path1)


In [2]:
##IMPRESSION DISCOUNTING

#dummy = URM_interactions

#Impressions is a string of numbers seperated by ",". So I am using str.split to convert them into a list of numbers(ItemIDs)
dummy["Impressions"] = dummy["Impressions"].str.split(",")

#Filling the NaN values with a dummy number that does not exist in ItemID list, these will be dropped later
dummy["Impressions"] = dummy["Impressions"].fillna(99999)

#Explode basicly explodes the list of Impressions, every number in the list is seperated intro independent rows with 
#corresponding UserID ItemID and Data
URM_impressions = dummy.explode(["Impressions"])

#Dropping the NaN values, we assigned them to dummy numbers otherwise explode does not work 
URM_impressions = URM_impressions.drop(URM_impressions[URM_impressions["Impressions"] == 99999].index)

#Dropping the itemID = Impression, meaning the item user interacted amongst the impression list
URM_impressions = URM_impressions[URM_impressions["ItemID"] != URM_impressions["Impressions"]]

#Now every impression is a seperate itemID that I can assign ratings. So I am selecting those columns and renaming Impressions
#as my new ItemID
URM_impressions = URM_impressions[["UserID","Impressions","Data"]]
URM_impressions.rename(columns = {'Impressions':'ItemID'}, inplace = True)

#I assign every Impression a rating of -1, meaning if the user did not interacted with the item, user did not like it
URM_impressions["Data"] = -1

URM_impressions.dtypes

UserID    object
ItemID    object
Data       int64
dtype: object

In [3]:
def precision(recommended_items, relevant_items):
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    precision_score = np.sum(is_relevant, dtype=np.float32) / len(is_relevant)

    return precision_score


def recall(recommended_items, relevant_items):
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    recall_score = np.sum(is_relevant, dtype=np.float32) / relevant_items.shape[0]

    return recall_score


def ap(recommended_items, relevant_items):
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))

    ap_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return ap_score

In [4]:
"""interactions_and_impressions.csv : Contains the training set, describing implicit preferences expressed by the users.
-user_id : identifier of the user
-item_id : identifier of the item (TV series)
-impression_list : string containing the items that were present on the screen when the user interacted with the 
 item in column item_id. Not all interactions have a corresponding impressions list.
-data : "0" if the user watched the item, "1" if the user opened the item details page."""
URM_interactions

,UserID,ItemID,Impressions,Data
0,0,11,"0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19",1
1,0,21,NaN,0
2,0,21,NaN,0
3,0,21,"20,21,22,23,24,25,26,27,28,29",0
4,0,21,NaN,1
...,...,...,...,...
5826501,41628,20448,NaN,0
5826502,41628,20896,NaN,1
5826503,41628,21506,NaN,1
5826504,41628,22882,NaN,0


In [5]:
URM_interactions.describe()

,UserID,ItemID,Data
count,5.826506e+06,5.826506e+06,5.826506e+06
mean,1.899451e+04,4.641730e+03,3.877156e-01
std,1.117126e+04,6.533349e+03,4.872291e-01
min,0.000000e+00,0.000000e+00,0.000000e+00
25%,9.315000e+03,3.370000e+02,0.000000e+00
50%,1.893600e+04,1.448000e+03,0.000000e+00
75%,2.831300e+04,6.016000e+03,1.000000e+00
max,4.162800e+04,2.450600e+04,1.000000e+00


In [6]:
URM_interactions = URM_interactions[["UserID","ItemID","Data"]]
URM_interactions = pd.concat([URM_interactions,URM_impressions])

watched = 5;
open_descr = 3;

URM_interactions['Data'][URM_interactions['Data'] == 1] = open_descr # opened - interested
URM_interactions['Data'][URM_interactions['Data'] == 0] = watched # watched

C:\Users\Utente\AppData\Local\Temp\ipykernel_21088\1838698791.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  URM_interactions['Data'][URM_interactions['Data'] == 1] = open_descr # opened - interested
C:\Users\Utente\AppData\Local\Temp\ipykernel_21088\1838698791.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  URM_interactions['Data'][URM_interactions['Data'] == 0] = watched # watched


In [7]:
URM_interactions[URM_interactions["UserID"] == 0]

,UserID,ItemID,Data
0,0,11,3
1,0,21,5
2,0,21,5
3,0,21,5
4,0,21,3
...,...,...,...
152,0,22307,3
153,0,23170,3
154,0,23214,3
155,0,24264,3


In [8]:
User_Episode = URM_interactions[["UserID","ItemID","Data"]][URM_interactions['Data'] == watched]
User_Episode["Data"] = User_Episode.groupby(["UserID","ItemID"])["ItemID"].transform("count")
User_Episode.rename(columns = {'Data':'Watched'}, inplace = True)
User_Episode.reset_index()

ICM_length.rename(columns = {'item_id':'ItemID'}, inplace = True)

User_Episode = User_Episode.merge(ICM_length,on = "ItemID",how = "outer")
User_Episode = User_Episode.dropna(subset = ["UserID","Watched","data"])

User_Episode["Watched"]/=User_Episode["data"] + 1

User_Episode = User_Episode[["UserID", "ItemID","Watched"]]
User_Episode = User_Episode.drop_duplicates(subset = ["UserID" , "ItemID"])

User_Episode

C:\Users\Utente\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)


,UserID,ItemID,Watched
0,0,21.0,0.107143
3,14,21.0,0.035714
4,15,21.0,0.071429
6,16,21.0,0.035714
7,25,21.0,0.035714
...,...,...,...
3567463,25890,21018.0,0.055556
3567464,26744,21018.0,0.055556
3567465,31457,21018.0,0.055556
3567466,32594,21018.0,0.055556


In [9]:
URM_interactions = URM_interactions.merge(User_Episode,on = ["UserID","ItemID"] ,how = "outer")
URM_interactions[URM_interactions["UserID"] == 0]
#URM_interactions["Watched"] = URM_interactions[["Watched"]].fillna(1.0)
#URM_interactions["Data"]*=URM_interactions["Watched"]


#URM_interactions = URM_interactions.groupby(["UserID","ItemID"])["Data"].mean()
#URM_interactions = URM_interactions.reset_index()

#URM_interactions[URM_interactions["UserID"] == 0]

,UserID,ItemID,Data,Watched
0,0,11,3,NaN
1,0,21,5,0.107143
2,0,21,5,0.107143
3,0,21,5,0.107143
4,0,21,3,0.107143
...,...,...,...,...
152,0,22307,3,NaN
153,0,23170,3,NaN
154,0,23214,3,NaN
155,0,24264,3,NaN


In [11]:
URM_interactions["Watched"] = URM_interactions[["Watched"]].fillna(1.0)

In [15]:
URM_interactions["Data"]*=URM_interactions["Watched"]

In [18]:
URM_interactions = URM_interactions.groupby(["UserID","ItemID"])["Data"].mean()

In [21]:
URM_interactions = URM_interactions.reset_index()

In [24]:
URM_interactions[URM_interactions["UserID"] == 0]

,UserID,ItemID,Data
0,0,11,3.000
1,0,21,0.375
2,0,22,3.000
3,0,24,3.000
4,0,44,3.000
...,...,...,...
61,0,22307,3.000
62,0,23170,3.000
63,0,23214,3.000
64,0,24264,3.000


In [ ]:
URM_interactions[URM_interactions["UserID"] == 1]

In [ ]:
ICM_length

In [ ]:
"""data_ICM_length.csv :
Contains the number of episodes of the items. TV series may have multiple episodes.

-item_id : identifier of the item
-feature_id : identifier of the feature, only one value (0) exists since this ICM only contains the feature "length"
-data : number of episodes. Some values may be 0 due to incomplete data."""
ICM_length.drop(columns = 'feature_id', inplace=True) # same feature is not needed
ICM_length 

In [ ]:
ICM_length["data"] = ICM_length["data"].fillna(0)
ICM_length.loc[ICM_length["data"] > 1 , "data"] = 10 #movie
ICM_length.loc[ICM_length["data"] == 1 , "data"] = 11 #series
ICM_length = ICM_length.drop(ICM_length[ICM_length.data == 0].index)
ICM_length.rename(columns = {'data':'feature_id'}, inplace = True)
ICM_length

In [ ]:
ICM_length.describe()

In [ ]:
"""data_ICM_type.csv:
Contains the type of the items. An item can only have one type.
All types are anonymized and described only by a numerical identifier.

-item_id : identifier of the item
-feature_id : identifier of the type
-data : "1" if the item is described by the type"""
ICM_type.drop(columns = 'data', inplace=True) # same feature is not needed
ICM_type

In [ ]:
#print ("The number of interactions is {}".format(len(URM_interactions)))
#print ("The number of users is {}".format(max(URM_interactions['UserID'])+1))
#print ("The number of items is {}".format(max(ICM_length['ItemID'])+1))
#print("Sparcity: {}%".format(1-len(int_and_impr)/(n_users*n_items)))

In [ ]:
ICM_type.rename(columns = {'item_id':'ItemID'}, inplace = True)
#ICM_length.rename(columns = {'item_id':'ItemID'}, inplace = True)
ICM_type = ICM_type.append(ICM_length, ignore_index=True)
ICM_type

In [ ]:
#ICM_type.rename(columns = {'data':'epNum'}, inplace = True)

In [ ]:
ICM = URM_interactions.merge(ICM_type, on="ItemID", how="outer" )

ICM

In [ ]:
ICM = ICM.dropna(subset=['UserID'])
ICM["feature_id"] = ICM["feature_id"].fillna(0)
ICM

In [ ]:
URM_interactions.dtypes

In [ ]:
#ICM["epNum"] = ICM["epNum"].fillna(0)
#ICM

In [ ]:
mapped_id, original_id = pd.factorize(URM_interactions["UserID"].unique())

print("Unique UserID in the URM are {}".format(len(original_id)))

all_item_indices = pd.concat([URM_interactions["UserID"], ICM["UserID"],], ignore_index=True)
mapped_id, original_id = pd.factorize(all_item_indices.unique())

print("Unique UserID in the URM and ICM are {}".format(len(original_id)))

user_original_ID_to_index = pd.Series(mapped_id, index=original_id)

In [ ]:
mapped_id, original_id = pd.factorize(URM_interactions["ItemID"].unique())

print("Unique ItemID in the URM are {}".format(len(original_id)))
all_item_indices = pd.concat([URM_interactions["ItemID"], ICM["ItemID"]], ignore_index=True)
mapped_id, original_id = pd.factorize(all_item_indices.unique())

print("Unique ItemID in the URM and ICM are {}".format(len(original_id)))

item_original_ID_to_index = pd.Series(mapped_id, index=original_id)

In [ ]:
mapped_id, original_id = pd.factorize(URM_interactions["Data"].unique())
data_original_ID_to_index = pd.Series(mapped_id, index=original_id)

In [ ]:
mapped_id, original_id = pd.factorize(ICM["feature_id"].unique())
feature_original_ID_to_index = pd.Series(mapped_id, index=original_id)

In [ ]:
#mapped_id, original_id = pd.factorize(ICM["epNum"].unique())
#epNum_original_ID_to_index = pd.Series(mapped_id, index=original_id)

In [ ]:
URM_interactions["ItemID"] = URM_interactions["ItemID"].map(item_original_ID_to_index)
URM_interactions["UserID"] = URM_interactions["UserID"].map(user_original_ID_to_index)
URM_interactions["Data"] = URM_interactions["Data"].map(data_original_ID_to_index)

In [ ]:
ICM["ItemID"] = ICM["ItemID"].map(item_original_ID_to_index)
ICM["UserID"] = ICM["UserID"].map(user_original_ID_to_index)
#ICM["epNum"] = ICM["epNum"].map(epNum_original_ID_to_index)
ICM["feature_id"] = ICM["feature_id"].map(feature_original_ID_to_index)


ICM



In [ ]:
n_users = len(user_original_ID_to_index)
n_features = len(feature_original_ID_to_index)
n_items = len(item_original_ID_to_index)
#n_episode = len(epNum_original_ID_to_index)


In [ ]:
#CREATE URM

URM_all = sps.coo_matrix((np.ones(len(URM_interactions["UserID"].values)), 
                          (URM_interactions["UserID"].values, URM_interactions["ItemID"].values)), 
                         shape = (n_users, n_items))# change the structure here
URM_all

In [ ]:
#CREATE ICM
ICM_all = sps.csr_matrix((np.ones(len(ICM["ItemID"].values)), 
                          (ICM["ItemID"].values, ICM["feature_id"].values)),
                        shape = (n_items, n_features))

ICM_all.data = np.ones_like(ICM_all.data)

ICM_all

In [ ]:
ICM_all = sps.csr_matrix(ICM_all)
features_per_item = np.ediff1d(ICM_all.indptr)

ICM_all = sps.csc_matrix(ICM_all)
items_per_feature = np.ediff1d(ICM_all.indptr)

ICM_all = sps.csr_matrix(ICM_all)

features_per_item = np.sort(features_per_item)
items_per_feature = np.sort(items_per_feature)

import matplotlib.pyplot as pyplot
%matplotlib inline  

pyplot.plot(features_per_item, 'ro')
pyplot.ylabel('Num features ')
pyplot.xlabel('Sorted items')
pyplot.show()

pyplot.plot(items_per_feature, 'ro')
pyplot.ylabel('Num items ')
pyplot.xlabel('Sorted features')
pyplot.show()

In [ ]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.85)


In [ ]:
def linear_confidence_function(URM_train, alpha):
    
    URM_train.data = 1.0 + alpha*URM_train.data
    
    return URM_train

def popularity_confidence(URM_train):
    
    item_popularity = np.ediff1d(URM_train.tocsc().indptr)
    item_confidence = np.zeros(len(item_popularity))
    item_confidence[item_popularity!=0] = np.log(item_popularity[item_popularity!=0])
    
    C_URM_train = URM_train.copy()
    C_URM_train = C_URM_train.tocsc()
    
    for item_id in range(C_URM_train.shape[1]):
        start_pos = C_URM_train.indices[item_id]
        end_pos = C_URM_train.indices[item_id+1]
        
        C_URM_train.data[start_pos:end_pos] = item_confidence[item_id]
    
    C_URM_train = C_URM_train.tocsr()
    
    return C_URM_train

def _update_row(interaction_profile, interaction_confidence, Y, YtY, regularization_diagonal):

    Y_interactions = Y[interaction_profile, :]
    
    A = Y_interactions.T.dot(((interaction_confidence - 1) * Y_interactions.T).T)

    B = YtY + A + regularization_diagonal

    return np.dot(np.linalg.inv(B), Y_interactions.T.dot(interaction_confidence))

In [ ]:
C_URM_train = popularity_confidence(URM_train)

In [ ]:
regularization_coefficient = 1e-4
num_factors = 10

regularization_diagonal = np.diag(regularization_coefficient * np.ones(num_factors))

In [ ]:
import time
C_URM_train_csc = C_URM_train.tocsc()

num_factors = 10

user_factors = np.random.random((n_users, num_factors))
item_factors = np.random.random((n_items, num_factors))


for n_epoch in range(10):
    
    start_time = time.time()

    VV = item_factors.T.dot(item_factors)
        
    for user_id in range(C_URM_train.shape[0]):

        start_pos = C_URM_train.indptr[user_id]
        end_pos = C_URM_train.indptr[user_id + 1]

        user_profile = C_URM_train.indices[start_pos:end_pos]
        user_confidence = C_URM_train.data[start_pos:end_pos]
        
        user_factors[user_id, :] = _update_row(user_profile, user_confidence, item_factors, VV, regularization_diagonal)   

        # Print some stats
        if (user_id +1)% 100000 == 0 or user_id == C_URM_train.shape[0]-1:
            elapsed_time = time.time() - start_time
            samples_per_second = user_id/elapsed_time
            print("Iteration {} in {:.2f} seconds. Users per second {:.2f}".format(user_id+1, elapsed_time, samples_per_second))
    
    UU = user_factors.T.dot(user_factors)

    for item_id in range(C_URM_train.shape[1]):

        start_pos = C_URM_train_csc.indptr[item_id]
        end_pos = C_URM_train_csc.indptr[item_id + 1]

        item_profile = C_URM_train_csc.indices[start_pos:end_pos]
        item_confidence = C_URM_train_csc.data[start_pos:end_pos]

        item_factors[item_id, :] = _update_row(item_profile, item_confidence, user_factors, UU, regularization_diagonal)    

        # Print some stats
        if (item_id +1)% 100000 == 0 or item_id == C_URM_train.shape[1]-1:
            elapsed_time = time.time() - start_time
            samples_per_second = item_id/elapsed_time
            print("Iteration {} in {:.2f} seconds. Items per second {:.2f}".format(item_id+1, elapsed_time, samples_per_second))

    total_epoch_time = time.time() - start_time  
    print("Epoch {} complete in in {:.2f} seconds".format(n_epoch+1, total_epoch_time))

In [ ]:
item_factors = np.transpose(item_factors)
test = user_factors*item_factors 


In [ ]:
from Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])


In [ ]:
#results_df, _ = evaluator_test.evaluateRecommender(itemKNNCF)
#results_df

In [ ]:

from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {
    "topK": Integer(5, 1000),
    "shrink": Integer(0, 1000),
    "similarity": Categorical(["cosine"]),
    "normalize": Categorical([True, False]),
}

from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

recommender_class = UserKNNCFRecommender

hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                         evaluator_validation=evaluator_validation,
                                         evaluator_test=evaluator_test)

from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
  
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},
    EARLYSTOPPING_KEYWORD_ARGS = {},
)

recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train_validation],     # For a CBF model simply put [URM_train_validation, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},
    EARLYSTOPPING_KEYWORD_ARGS = {},
)

import os

output_folder_path = "result_experiments/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 10  # using 10 as an example
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

hyperparameterSearch.search(recommender_input_args,
                       recommender_input_args_last_test = recommender_input_args_last_test,
                       hyperparameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       save_model = "last",
                       output_folder_path = output_folder_path, # Where to save the results
                       output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
                       metric_to_optimize = metric_to_optimize,
                       cutoff_to_optimize = cutoff_to_optimize,
                      )


In [ ]:
from Recommenders.DataIO import DataIO

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")

search_metadata.keys()

In [ ]:
hyperparameters_df = search_metadata["hyperparameters_df"]
hyperparameters_df

In [ ]:
result_on_validation_df = search_metadata["result_on_validation_df"]
result_on_validation_df

In [ ]:
result_best_on_test = search_metadata["result_on_last"]
result_best_on_test

In [ ]:
best_hyperparameters_ItemKNNCF = {'topK': 816, 'shrink': 0, 'similarity': 'cosine', 'normalize': True}
best_hyperparameters = search_metadata["hyperparameters_best"]
best_hyperparameters

In [ ]:
#best_hyperparams_ItemKNNCF = best_hyperparameters
best_hyp = {'topK': 543, 'shrink': 360, 'similarity': 'cosine', 'normalize': True}
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
user = ItemKNNCFRecommender(URM_train)
user.fit(**best_hyp)

results_df, _ = evaluator_test.evaluateRecommender(user)
results_df

In [ ]:
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
best_hyp = {'topK': 543, 'shrink': 360, 'similarity': 'cosine', 'normalize': True}
urm_train_validation = URM_train + URM_validation
best_recommender = UserKNNCFRecommender(URM_all)
best_recommender.fit(**best_hyp)


In [ ]:
recommender_object = ItemKNN_CFCBF_Hybrid_Recommender(URM_all,ICM_all)
recommender_object

In [ ]:
recommender_object.load_model(output_folder_path, 
                              file_name = recommender_object.RECOMMENDER_NAME + "_best_model_last.zip" )

In [ ]:
path4 = path + '/data_target_users_test.csv'
users_to_recommend = pd.read_csv(path4)
users_to_recommend

In [ ]:

import scipy.sparse as sp
def prepare_submission(ratings: pd.DataFrame, users_to_recommend: np.array, urm_train: sp.csr_matrix, recommender: object):
    users_ids_and_mappings = user_original_ID_to_index.to_frame()
    users_ids_and_mappings = users_ids_and_mappings.drop_duplicates()
    users_ids_and_mappings = users_ids_and_mappings.reset_index(level=0)
    users_ids_and_mappings.columns = ["mapped_user_id","user_id"]
    users_ids_and_mappings = users_ids_and_mappings[users_ids_and_mappings.user_id.isin(users_to_recommend["user_id"])][["mapped_user_id", "user_id"]].drop_duplicates()
    
    items_ids_and_mappings = item_original_ID_to_index.to_frame()
    items_ids_and_mappings = items_ids_and_mappings.drop_duplicates()
    items_ids_and_mappings = items_ids_and_mappings.reset_index(level=0)
    items_ids_and_mappings.columns = ["mapped_item_id","item_id"]
    
    inv_map = dict(item_original_ID_to_index)
    mapping_to_item_id = {v: k for k, v in inv_map.items()}
    
    
    recommendation_length = 10
    submission = []
    for idx, row in users_ids_and_mappings.iterrows():
        user_id = row.user_id
        mapped_user_id = row.mapped_user_id
        
        recommendations = recommender.recommend(int(mapped_user_id),10)
        print(user_id ,",", *recommendations)
        rec_list = [mapping_to_item_id[ItemID] for ItemID in recommendations]
        submission.append((user_id, rec_list))
        
    return submission


In [ ]:
submission = prepare_submission(URM_all, users_to_recommend, URM_train_validation, user)


In [ ]:
submission


In [ ]:
def write_submission(submissions):
    with open("./submission.csv", "w") as f:
        f.write("user_id,item_list\n")
        for user_id, items in submissions:
            f.write(f"{user_id},{' '.join([str(item) for item in items])}\n")

In [ ]:
write_submission(submission)